<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import pulp

In [2]:
g7=pd.read_csv('G7 Distances.csv')

In [3]:
g7.head(7)

,Capital,Country,latitude,longitude,Ottawa,Paris,Berlin,Rom,Tokyo,London,Washington
0,Ottawa,Canada,45.421106,-75.690308,0,5648,6127,6728,10319,5360,734
1,Paris,France,48.856697,2.351462,5648,0,876,1105,9715,343,6165
2,Berlin,Germany,52.517037,13.388860,6127,876,0,1183,8920,930,6710
3,Rom,Italy,41.893320,12.482932,6728,1105,1183,0,9859,1433,7217
4,Tokyo,Japan,35.682839,139.759455,10319,9715,8920,9859,0,9561,10902
5,London,UK,51.507322,-0.127647,5360,343,930,1433,9561,0,5898
6,Washington,USA,38.894985,-77.036571,734,6165,6710,7217,10902,5898,0


In [4]:
n = len(g7)
capitals = list(g7['Capital'])

In [5]:
capitals

['Ottawa', 'Paris', 'Berlin', 'Rom', 'Tokyo', 'London', 'Washington']

In [6]:
def distance(i, j):
    return g7.loc[i][capitals[j]]

In [7]:
distance(0, 1)

5648

Define a set of variables

In [8]:
x = pulp.LpVariable.dicts("x", ( range(0,n), range(0,n) ),
                          lowBound=0, upBound=1, 
                          cat=pulp.LpInteger)

Define the problem

In [9]:
prob = pulp.LpProblem("TSPG7",pulp.LpMinimize)

Define the objective function

In [10]:
prob += pulp.lpSum([ distance(i,j)*x[i][j] 
                        for i in range(0,n)
                            for j in range(0,n) 
                                if i!=j
                   ])

Define the Constraints

Every town was entered only once:

In [11]:
for j in range(0,n):
    prob += pulp.lpSum([ x[i][j] 
                            for i in range(0,n) 
                                if i!=j
                       ]) ==1

Every town was left only once:

In [12]:
for i in range(0,n):
    prob += pulp.lpSum([ x[i][j] 
                            for j in range(0,n) 
                                if i!=j
                       ]) ==1

There are no sub-cycles of length k < n/2

In [13]:
def cycles(k, n):
    if k==1:
        return [ [i] for i in range(0,n) ]
    else:
        sc=cycles(k-1, n)
        all=[]
        for c in sc:
            for i in range(0,n):
                if c.count(i)==0:
                    all.append(c+[i])
        return all

In [14]:
for k in range(2,n//2+1):
    cycs=cycles(k,n)
    for c in cycs:
        c.append(c[0])
        prob+=pulp.lpSum([ x[c[i]][c[i+1]] for i in range(0,k)]) <= k-1

In [16]:
prob.solve()

1

In [17]:
start = capitals.index('Berlin')
trip = [ capitals[start] ]
i = start
while len(trip)<n:
    for j in range(0, n):
        if pulp.value(x[i][j])==1:
            trip.append(capitals[j])
            i=j
            break
trip.append(capitals[start])
trip

['Berlin', 'Tokyo', 'Ottawa', 'Washington', 'London', 'Paris', 'Rom', 'Berlin']

In [18]:
pulp.value(prob.objective)

28502.0